# Channel Pruning in PyTorch

A quick guide on how to use the script to compress your own models. You will need to write your own `main` script that includes the model you have and all the data processing - this should serve as a good example of how to do it. 

We'll use LeNet for now since it can safely be trained/retrained in the browser without too much effort.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import numpy as np

from models import *
from utils  import *
from torch.autograd import Variable

# get the data 
# you will need to have CIFAR-10 in ./data for this tutorial
train_loader, test_loader = get_data() # <- this is in the utils file

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


## Initial training cycle

To save space we will skip the initial training cycle here and just load a pre-trained model, which is included in the GitHub repository. If you're training your own model then you can just replace this with your own standard training process

In [ ]:
base_model, base_model_name = LeNet(), "LeNet"
compressed_model = LeNetCompressed

'''
Iterative sparsity. Han's paper mentions that better compression rates are achieved
when you incrementally sparsify the model, so that's what we do here.
'''

finetuning_epochs = 10

base_model_name, base_model, best_acc = load_best(base_model_name, base_model)
base_model_name = "LeNetSparse"

for sparsity in [50., 60., 70., 80., 90.]:
    
    print("\n\n\n\n===== ", sparsity, " =====\n\n")
    # load the pretrained model
        
    best_acc = 0.

    # sparsify
    model_weights = sparsify(base_model, sparsity)

    # train with 0.01
    best_acc = finetune(model_weights, base_model_name, best_acc, finetuning_epochs, train_loader, test_loader, 0.01)
    # train with 0.001
    best_acc = finetune(model_weights, base_model_name, best_acc, finetuning_epochs, train_loader, test_loader, 0.001)


new_model = compress_convs(model_weights, compressed_model)
new_model_name = base_model_name + "Compressed"

# finetune again - this is just to save the model
finetune(new_model, new_model_name, 0., 10, train_loader, test_loader, 0.001)


Loading checkpoint with best_acc:  68.87




=====  50.0  =====



Epoch: 1
     Accuracy:  65.246
Saving..
==> Saving model ...

Epoch: 2
     Accuracy:  65.776

Epoch: 3
     Accuracy:  65.666

Epoch: 4
     Accuracy:  65.88

Epoch: 5
     Accuracy:  66.32

Epoch: 6
     Accuracy:  66.302

Epoch: 7
     Accuracy:  65.83

Epoch: 8
     Accuracy:  66.312

Epoch: 9
     Accuracy:  66.768

Epoch: 1
     Accuracy:  69.182
Saving..
==> Saving model ...

Epoch: 2
     Accuracy:  69.89

Epoch: 3
     Accuracy:  70.2
Saving..
==> Saving model ...

Epoch: 4
     Accuracy:  70.474

Epoch: 5
     Accuracy:  70.518

Epoch: 6
     Accuracy:  70.586
Saving..
==> Saving model ...

Epoch: 7
     Accuracy:  70.736
Saving..
==> Saving model ...

Epoch: 8
     Accuracy:  70.846
Saving..
==> Saving model ...

Epoch: 9
     Accuracy:  70.748




=====  60.0  =====



Epoch: 1
     Accuracy:  65.162
Saving..
==> Saving model ...

Epoch: 2
     Accuracy:  66.486
Saving..
==> Saving model ...

Epoch: 3
     